In [29]:
import pandas as pd
import numpy as np
import sklearn as sk

In [2]:
df = pd.read_csv("C:\Users\karagiag\Downloads\^GSPC.csv", parse_dates = True, index_col=0)

In [3]:
df = df.rename(columns={"Adj Close":"price"})

In [4]:
df.head()

,Open,High,Low,Close,price,Volume
Date,,,,,,
2001-01-10,1300.800049,1313.760010,1287.280029,1313.270020,1313.270020,1296500000
2001-01-11,1313.270020,1332.189941,1309.719971,1326.819946,1326.819946,1411200000
2001-01-12,1326.819946,1333.209961,1311.589966,1318.550049,1318.550049,1276000000
2001-01-16,1318.319946,1327.810059,1313.329956,1326.650024,1326.650024,1205700000
2001-01-17,1326.650024,1346.920044,1325.410034,1329.469971,1329.469971,1349100000


In [5]:
df.drop(df.columns[[0,1,2,3,5]],axis=1, inplace=True)

In [6]:
df.head()

,price
Date,
2001-01-10,1313.270020
2001-01-11,1326.819946
2001-01-12,1318.550049
2001-01-16,1326.650024
2001-01-17,1329.469971


In [7]:
# Create lagged time series (2 lags)
lags=2
for i in range(0,lags):
    df["Lag %s" % str(i+1)] = df['price'].shift(i+1)

In [8]:
df.head()

,price,Lag 1,Lag 2
Date,,,
2001-01-10,1313.270020,NaN,NaN
2001-01-11,1326.819946,1313.270020,NaN
2001-01-12,1318.550049,1326.819946,1313.270020
2001-01-16,1326.650024,1318.550049,1326.819946
2001-01-17,1329.469971,1326.650024,1318.550049


In [9]:
# Calculate S&P daily log returns
df['ret'] = np.log(df['price']/df['price'].shift(1))

In [14]:
# Create lagged retunrs (2 lags)
lags=2
for i in range(0,lags):
    df["ret-%s" % str(i+1)] =np.log(df['Lag %s' %str(i+1)]/ df['Lag %s' %str(i+1)].shift(1))

In [15]:
#Create positions based on return sign
df['position'] = np.sign(df['ret'])

In [16]:
df.head()

,price,Lag 1,Lag 2,ret,ret-1,ret-2,position
Date,,,,,,,
2001-01-10,1313.270020,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-11,1326.819946,1313.270020,NaN,0.010265,NaN,NaN,1.0
2001-01-12,1318.550049,1326.819946,1313.270020,-0.006252,0.010265,NaN,-1.0
2001-01-16,1326.650024,1318.550049,1326.819946,0.006124,-0.006252,0.010265,1.0
2001-01-17,1329.469971,1326.650024,1318.550049,0.002123,0.006124,-0.006252,1.0


In [48]:
df=df.dropna()

In [49]:
# Use the prior two days of returns as predictor values
X= df[["ret-1","ret-2"]]

In [83]:
y=df['position']

In [51]:
import datetime

In [52]:
# Create training and test sets (before and after 1/1/2005)
start_test = datetime.datetime(2005,1,1)
X_train = X[X.index < start_test]
X_test = X[X.index >= start_test]
y_train = y[y.index < start_test]
y_test = y[y.index >= start_test]

In [53]:
from sklearn.linear_model import LogisticRegression

In [54]:
model=LogisticRegression()

In [77]:
results=model.fit(X_train,y_train)

In [58]:
y_pred=model.predict(X_test)

In [79]:
print results.params

AttributeError: 'LogisticRegression' object has no attribute 'params'

In [59]:
confusion_matrix(y_pred,y_test)

array([[   0,    0,    0],
       [   0,    0,    0],
       [1258,    1, 1509]])

In [67]:
print("%s\n" % confusion_matrix(y_test,y_pred,))

[[   0    0 1258]
 [   0    0    1]
 [   0    0 1509]]



In [ ]:
print("%s:\n%0.3f" % (m[0], model.score(X_test, y_test)))

In [ ]:
#plot logistic regression

In [73]:
models= [("LR", LogisticRegression()), 
        ("LDA", LDA()),
        ("QDA", QDA()),
        ("LSVC", LinearSVC()),
        ("RSVM", SVC())]

In [75]:
for m in models:
    print m[1]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)
QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
               store_covariances=False, tol=0.0001)
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [26]:
from sklearn.metrics import confusion_matrix

In [66]:
help(confusion_matrix)

Help on function confusion_matrix in module sklearn.metrics.classification:

confusion_matrix(y_true, y_pred, labels=None)
    Compute confusion matrix to evaluate the accuracy of a classification
    
    By definition a confusion matrix :math:`C` is such that :math:`C_{i, j}`
    is equal to the number of observations known to be in group :math:`i` but
    predicted to be in group :math:`j`.
    
    Read more in the :ref:`User Guide <confusion_matrix>`.
    
    Parameters
    ----------
    y_true : array, shape = [n_samples]
        Ground truth (correct) target values.
    
    y_pred : array, shape = [n_samples]
        Estimated targets as returned by a classifier.
    
    labels : array, shape = [n_classes], optional
        List of labels to index the matrix. This may be used to reorder
        or select a subset of labels.
        If none is given, those that appear at least once
        in ``y_true`` or ``y_pred`` are used in sorted order.
    
    Returns
    -------
    

In [72]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.lda import LDA
from sklearn.metrics import confusion_matrix
from sklearn.qda import QDA
from sklearn.svm import LinearSVC, SVC

C:\Program Files\Continuum Analytics\Anaconda2\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Program Files\Continuum Analytics\Anaconda2\lib\site-packages\sklearn\qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)
